# sharp ratio maximize

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime,timedelta
from dateutil.relativedelta import relativedelta
from collections import Counter
import calendar
import random
import warnings
import functools

from matplotlib import pyplot as plt
import matplotlib.font_manager as fm

In [2]:
def make_simul_port(len=5,num=2000):
    list_simul = []
    for i in range(num):
        tmp = random.sample(range(1, 100), len)
        weight = tmp/np.sum(tmp)
        list_simul.append(weight)
    return(list_simul)

In [3]:
def shape_portpolio(next_day,simul = 2000):

    tmp = asset_factor[asset_factor.date==str(next_day)[0:10]].iloc[0][1:]
    next_list = list(tmp[tmp.notnull()].index)
    weight_list = make_simul_port(len=len(next_list),num=simul)
    tmp_price = change_price2[change_price2.date < next_day]
    weight_df = pd.DataFrame(weight_list,columns=next_list)

    # simulation
    port_volatility_list = []
    portfolioSimpleAnuualReturn_list = []
    sharpe_ratio_list = []

    for i in weight_list:    
        cov_matric_annual = tmp_price[next_list].cov() * 252
        port_variance = np.dot(i, np.dot(cov_matric_annual,i))
        port_volatility = np.sqrt(port_variance)

        portfolioSimpleAnuualReturn = np.sum(tmp_price[next_list].mean() * i) * 252
        sharpe_ratio = portfolioSimpleAnuualReturn/port_volatility

        port_volatility_list.append(port_volatility)
        portfolioSimpleAnuualReturn_list.append(portfolioSimpleAnuualReturn)
        sharpe_ratio_list.append(sharpe_ratio)

    weight_df['risk'] = port_volatility_list
    weight_df['return'] = portfolioSimpleAnuualReturn_list
    weight_df['sharpe'] = sharpe_ratio_list
    max_sharpe = weight_df.loc[weight_df['sharpe'] == weight_df['sharpe'].max()]
    min_risk = weight_df.loc[weight_df['risk'] == weight_df['risk'].min()]

    return (weight_df,max_sharpe, min_risk)

In [4]:
# next_day = datetime.strptime('2020-01-03','%Y-%m-%d')
# tmp_sharpe = shape_portpolio(next_day,simul = 20000)
# weight_df = tmp_sharpe[0]; max_sharpe = tmp_sharpe[1]; min_risk = tmp_sharpe[2]

In [5]:
# fig, ax = plt.subplots()
# weight_df.plot.scatter(x='risk', y='return', c='sharpe', cmap = 'viridis',edgecolors='k', grid=True, ax = ax)
# plt.scatter(x=min_risk['risk'], y=min_risk['return'], c='r', marker='X', s=200)
# plt.scatter(x=max_sharpe['risk'], y=max_sharpe['return'], c='r', marker='*', s=300)

# plt.title("Efficient Frontier")
# plt.ylabel('Expected Returns')
# plt.xlabel('Risk')
# plt.show()